In [ ]:
'''目标：
1，爬取豆瓣电影top250
2，爬取电影详情信息
3，爬取演员详情信息
'''

In [1]:
'''接口：
1,  TOP 250 	http://api.douban.com/v2/movie/top250 	start: 数据的开始项;count: 单页条数

'''

'接口：\n\n'

In [8]:
import json
import requests
import os
from tqdm import tqdm_notebook
import operator

In [73]:
#新建文件夹
if not os.path.exists('top250'):
    os.mkdir('top250')

In [124]:
#爬取全部250部电影
access_key = '0df993c66c0c636e29ecbb5344252a4a'  # api key
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
for i in tqdm_notebook(range(25)):
    url = f'https://api.douban.com/v2/movie/top250?apikey={access_key}&start={i*10}&count={10}'
    
    json_file_path = os.path.join('top250', '{}.json'.format(i))
    if os.path.exists(json_file_path):  #若已爬过，跳过
        continue  
        
    #防止超时，访问接口直到访问成功
    while True:
        try:
            response = requests.get(url,headers = headers,timeout = 5)
            break
        except requests.exceptions.Timeout:
            time.sleep(1)
            
    #写入文件
    with open(json_file_path,'w',encoding='utf-8') as f:
        f.write(json.dumps(json.loads(response.content.decode('utf-8')),indent=4,ensure_ascii=False))
        #indent=4:对齐 ensure_ascii=False:编码中文 json.dumps()用于将字典形式的数据转化为字符串，json.loads()用于将字符串形式的数据转化为字典

In [125]:
#将上面取的文件合并为top250.json
movies = []
for json_file in tqdm_notebook(os.listdir('top250')):
    json_file_path = os.path.join('top250', json_file)
    with open(json_file_path,'r',encoding = 'utf8') as f:
        
        movies += json.load(f)['subjects']

#按平均分降序排序
movies = sorted(movies,
               key = lambda movie: movie['rating']['average'],reverse= True)

#写入文件
with open('top250.json','w',encoding = 'utf8') as f:
    f.write(json.dumps({'subjects':movies},indent=4,ensure_ascii=False))
    

In [ ]:
#爬取演员数据（周星驰）

In [128]:
actor_id = '1048026'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
res = requests.get(f'http://api.douban.com/v2/movie/celebrity/{actor_id}?apikey={access_key}',headers=headers)
json.loads(res.content.decode('utf-8'))

{'website': '',
 'mobile_url': 'https://movie.douban.com/celebrity/1048026/mobile',
 'aka_en': ['Sing-Chi Chow (本名)', 'Sing Jai (昵称)'],
 'name': '周星驰',
 'works': [{'roles': ['演员'],
   'subject': {'rating': {'max': 10,
     'average': 9.2,
     'details': {'1': 1942.0,
      '3': 45618.0,
      '2': 3531.0,
      '5': 514550.0,
      '4': 201225.0},
     'stars': '45',
     'min': 0},
    'genres': ['喜剧', '爱情', '奇幻'],
    'title': '大话西游之大圣娶亲',
    'casts': [{'avatars': {'small': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p47421.jpg',
       'large': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p47421.jpg',
       'medium': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p47421.jpg'},
      'name_en': 'Stephen Chow',
      'name': '周星驰',
      'alt': 'https://movie.douban.com/celebrity/1048026/',
      'id': '1048026'},
     {'avatars': {'small': 'http://img3.doubanio.com/view/celebrity/s_ratio_celebrity/public/p45481.jpg',
  

In [1]:
#批量获取接口数据的方法
def get(url,headers):
    while True:
        try:
            res = requests.get(url,headers = headers, timeout = 5)
            return json.loads(res.content.decode('utf-8'))
        except requests.exceptions.Timeout:
            time.sleep(1)

In [6]:
import queue
import os
import requests
import json
access_key = '0df993c66c0c636e29ecbb5344252a4a'  # api key
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
# 演员队列初始为周星驰 id
actorQ = queue.Queue()
actorQ.put('1048026')

total = 100
# 创建文件夹方便整理数据
if not os.path.exists('actors'):
    os.mkdir('actors')

while not actorQ.empty():
    actor_id = actorQ.get()
    actor_path = os.path.join('actors', str(actor_id)+'.json')
    # 如果已爬取则从本地读取
    if os.path.exists(actor_path):
        # 读取文件
        with open(actor_path,encoding = 'utf-8') as f:
            json_content = json.load(f)
    # 否则爬取数据
    else:
        json_content = get(f'http://api.douban.com/v2/movie/celebrity/{actor_id}?apikey={access_key}',headers)
        # 爬完后写入本地
        with open(actor_path, 'w',encoding = 'utf-8') as f:
            f.write(json.dumps(json_content, indent=4, ensure_ascii=False))

    # 遍历该演员的电影的其他演员，并将 id 加入 actorQ 重新遍历
    for movie in json_content['works']:
        for x in movie['subject']['casts']:
            # 如果没有该演员的 id 则跳过
            if x['id'] == None:
                continue
            actorQ.put(x['id'])
    if len(os.listdir('actors')) > total:
        print('爬取完毕')
        break

爬取完毕


In [10]:

actors = {}
for json_file in tqdm_notebook(os.listdir('actors')):
    json_file_path = os.path.join('actors', json_file)
    with open(json_file_path,encoding = 'utf-8') as f:
        json_content = json.load(f)
        actors[json_content['id']] = json_content
# 写入文件
with open('actors.json', 'w',encoding = 'utf-8') as f:
    f.write(json.dumps({'actors': actors}, indent=4, ensure_ascii=False))    

In [11]:
#获取大量电影

In [14]:
#获取电影标签
tags = get('https://movie.douban.com/j/search_tags?type=movie&source=',headers)['tags']
tags

['热门',
 '最新',
 '经典',
 '可播放',
 '豆瓣高分',
 '冷门佳片',
 '华语',
 '欧美',
 '韩国',
 '日本',
 '动作',
 '喜剧',
 '爱情',
 '科幻',
 '悬疑',
 '恐怖',
 '成长']

In [19]:
#每一个标签下获取100部电影信息
import time
V = 100  # 尝试搜索次数，以获得更多结果
id_list = []


search_url = '\
https://movie.douban.com/j/search_subjects?type=movie&tag={}&sort=time&page_limit=20&page_start={}'


for tag in tags:
    print(f"开始搜索标签「{tag}」相关的电影 id")
    for i in tqdm_notebook(range(V)):
        url = search_url.format(tag, 20*i)
        json_content = get(url,headers)
        id_list += [x['id'] for x in json_content['subjects']]

# 去重，一部电影对应多个标签
id_list = list(set(id_list))

# 写入本地文件
with open('movie_id_list.json', 'w',encoding='utf-8') as f:
    f.write(json.dumps({'id': id_list}, indent=4, ensure_ascii=False))

开始搜索标签「热门」相关的电影 id


In [25]:
# 新建文件夹方便整理数据
if not os.path.exists('movies'):
    os.mkdir('movies')

# 读取电影 id 数据
with open('movie_id_list.json') as f:
    id_list = json.load(f)['id']

for movie_id in tqdm_notebook(id_list):
    url = f'https://api.douban.com/v2/movie/subject/{movie_id}?apikey={access_key}'
    json_path = os.path.join('movies', str(movie_id)+'.json')

    # 数据已爬取
    if os.path.exists(json_path):
        with open(json_path) as f:
            t = json.load(f)
            '''
            数据已爬取且没有出错，则跳过本条，例如返回:
            {
                "msg": "rate_limit_exceeded2: 125.70.79.50",
                "code": 112,
                "request": "GET /v2/movie/subject/26611090"
            }
            则是访问超过限制，重新爬取
            '''
            try:
                if t['code'] == 112:
                    pass
            except KeyError:
                continue
    json_content = get(url,headers)
    try:
        # 返回的数据为上面提起的错误，则跳过这条
        if json_content['code'] == 112:
            print('访问超过限制，请稍后再试')
            time.sleep(10)
    except KeyError:
        pass

    with open(json_path, 'w',encoding='utf-8') as f:
        f.write(json.dumps(json_content, indent=4, ensure_ascii=False))

In [28]:
movies = []
for json_file in tqdm_notebook(os.listdir('movies')):
    json_file_path = os.path.join('movies', json_file)
    with open(json_file_path,encoding='utf-8') as f:
        json_content = json.load(f)
        movies += [json_content]

# 写入文件
with open('movies.json', 'w',encoding='utf-8') as f:
    f.write(json.dumps({'movies': movies}, indent=4, ensure_ascii=False))